# 프로젝트 (1) moviepy로 비디오 처리하기

## 1 - 1. moviepy를 이용해서 주피터 노트북 상에서 비디오를 읽고 쓰는 프로그램을 작성해 주세요.

In [1]:
# 라이브러리 임포트

from moviepy.editor import VideoClip, VideoFileClip
from moviepy.editor import ipython_display
import cv2
import numpy as np
import os

In [2]:
# 샘플 동영상 읽기 및 저장

# 읽기
video_path = os.getenv('HOME')+'/aiffel/video_sticker_app/images/video2.mp4'
clip = VideoFileClip(video_path)
clip = clip.resize(width=640)
clip.ipython_display(fps=30, loop=True, autoplay=True, rd_kwargs=dict(logger=None))

# 쓰기
result_video_path = os.getenv('HOME')+'/aiffel/video_sticker_app/images/mvpyresult.mp4'
clip.write_videofile(result_video_path)

t:   0%|          | 0/404 [00:00<?, ?it/s, now=None]                

Moviepy - Building video /home/aiffel/aiffel/video_sticker_app/images/mvpyresult.mp4.
MoviePy - Writing audio in mvpyresultTEMP_MPY_wvf_snd.mp3
MoviePy - Done.
Moviepy - Writing video /home/aiffel/aiffel/video_sticker_app/images/mvpyresult.mp4



Moviepy - Done !
Moviepy - video ready /home/aiffel/aiffel/video_sticker_app/images/mvpyresult.mp4


## 1 - 2. moviepy 로 읽은 동영상을 numpy 형태로 변환하고 영상 밝기를 50% 어둡게 만든 후에 저장해 주세요.

In [3]:
# 샘플 동영상을 넘파이 형태로 및 밝기를 50% 어둡게 변환

# 읽기
video_path = os.getenv('HOME')+'/aiffel/video_sticker_app/images/video2.mp4'
clip = VideoFileClip(video_path)
clip = clip.resize(width=640)
clip.ipython_display(fps=30, loop=True, autoplay=True, rd_kwargs=dict(logger=None))

# clip 에서 numpy 로 데이터 추출
vlen = int(clip.duration*clip.fps)
video_container = np.zeros((vlen, clip.size[1], clip.size[0], 3), dtype=np.uint8)
for i in range(vlen):
    img = clip.get_frame(i/clip.fps)
    video_container[i] = (img * 0.5).astype(np.uint8)

# 새 clip 만들기
dur = vlen / clip.fps
outclip = VideoClip(lambda t: video_container[int(round(t*clip.fps))], duration=dur)

# 쓰기
result_video_path2 = os.getenv('HOME')+'/aiffel/video_sticker_app/images/mvpyresult2.mp4'
outclip.write_videofile(result_video_path2, fps=30)

t:  12%|█▏        | 48/403 [00:00<00:00, 473.27it/s, now=None]

Moviepy - Building video /home/aiffel/aiffel/video_sticker_app/images/mvpyresult2.mp4.
Moviepy - Writing video /home/aiffel/aiffel/video_sticker_app/images/mvpyresult2.mp4



Moviepy - Done !
Moviepy - video ready /home/aiffel/aiffel/video_sticker_app/images/mvpyresult2.mp4


## 1 - 3. 영상을 읽고 쓰는 시간을 측정해 주세요. OpenCV를 사용할 때와 차이를 측정해 보세요.

In [4]:
# 영상을 읽고 쓰는 시간을 측정 

# CASE 1 : moviepy 사용
start = cv2.getTickCount()
clip = VideoFileClip(video_path)
clip = clip.resize(width=640)

vlen = int(clip.duration*clip.fps)
video_container = np.zeros((vlen, clip.size[1], clip.size[0], 3), dtype=np.uint8)

for i in range(vlen):
    img = clip.get_frame(i/clip.fps)
    video_container[i] = (img * 0.5).astype(np.uint8)

dur = vlen / clip.fps
outclip = VideoClip(lambda t: video_container[int(round(t*clip.fps))], duration=dur)

mvpy_video_path = os.getenv('HOME')+'/aiffel/video_sticker_app/images/mvpyresult.mp4'
outclip.write_videofile(mvpy_video_path, fps=30)

time = (cv2.getTickCount() - start) / cv2.getTickFrequency()
print (f'[INFO] moviepy time : {time:.2f}ms')

t:  12%|█▏        | 48/403 [00:00<00:00, 475.85it/s, now=None]

Moviepy - Building video /home/aiffel/aiffel/video_sticker_app/images/mvpyresult.mp4.
Moviepy - Writing video /home/aiffel/aiffel/video_sticker_app/images/mvpyresult.mp4



Moviepy - Done !
Moviepy - video ready /home/aiffel/aiffel/video_sticker_app/images/mvpyresult.mp4
[INFO] moviepy time : 3.12ms


In [5]:
# 영상을 읽고 쓰는 시간을 측정 

# CASE 2 : OpenCV 사용
start = cv2.getTickCount()
vc = cv2.VideoCapture(video_path)

cv_video_path = os.getenv('HOME')+'/aiffel/video_sticker_app/images/cvresult.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
vw = cv2.VideoWriter(cv_video_path, fourcc, 30, (640,360))

vlen = int(vc.get(cv2.CAP_PROP_FRAME_COUNT))

for i in range(vlen):
    ret, img = vc.read()
    if ret == False: break
    
    img_result = cv2.resize(img, (640, 360)) * 0.5
    vw.write(img_result.astype(np.uint8))
    
time = (cv2.getTickCount() - start) / cv2.getTickFrequency()
print (f'[INFO] cv time : {time:.2f}ms')

[INFO] cv time : 1.66ms


## 1 - 4. moviepy 를 이용할 때의 장단점을 분석해 봅시다. 주피터 노트북에 답변을 작성해 코드와 함께 제출해 주세요.

OpenCV 를 통한 읽기 쓰기가 1.5 배 가량 빠른 속도를 보여주었다. 하지만 moviepy 를 이용한 결과물이 OpenCV 를 이용한 결과물보다 조금 더 넓은 화각을 가지고 있었다.

# 프로젝트 (2) 어디까지 만들고 싶은지 정의하기

## 2 -1. 실시간 카메라 스티커앱을 만들어봅시다.

In [1]:
# 웹캠 기반 실시간 카메라 스티커 앱 구현

import numpy as np
import cv2
import dlib

from newaddsticker import img2sticker

detector_hog = dlib.get_frontal_face_detector()
landmark_predictor = dlib.shape_predictor('./models/shape_predictor_68_face_landmarks.dat')

def main():
    cv2.namedWindow('show', 0)
    cv2.resizeWindow('show', 640, 360)

    vc = cv2.VideoCapture(0)   # 연결된 영상 장치의 인덱스, 하나만 있는 경우 0을 사용
    img_sticker = cv2.imread('./images/king.png')

    vlen = int(vc.get(cv2.CAP_PROP_FRAME_COUNT))
    print (vlen) # 웹캠은 video length 가 0 입니다.

    # 정해진 길이가 없기 때문에 while 을 주로 사용합니다.
    # for i in range(vlen):
    while True:
        ret, img = vc.read()
        if ret == False:
            break
        start = cv2.getTickCount()
        img = cv2.flip(img, 1)  # 보통 웹캠은 좌우 반전

        # 스티커 메소드를 사용
        img_result = img2sticker(img, img_sticker.copy(), detector_hog, landmark_predictor)   

        time = (cv2.getTickCount() - start) / cv2.getTickFrequency() * 1000
        print ('[INFO] time: %.2fms'%time)

        cv2.imshow('show', img_result)
        key = cv2.waitKey(1)
        if key == 27:
            break


if __name__ == '__main__':
    main()

-1
[INFO] time: 128.76ms
[INFO] time: 109.30ms
[INFO] time: 108.54ms
[INFO] time: 118.10ms
[INFO] time: 106.43ms
[INFO] time: 116.18ms
[INFO] time: 108.99ms
[INFO] time: 109.00ms
[INFO] time: 113.71ms
[INFO] time: 109.15ms
[INFO] time: 107.82ms
[INFO] time: 104.79ms
[INFO] time: 109.82ms
[INFO] time: 105.76ms
[INFO] time: 104.67ms
[INFO] time: 105.16ms
[INFO] time: 102.56ms
[INFO] time: 105.20ms
[INFO] time: 112.26ms
[INFO] time: 106.60ms
[INFO] time: 107.83ms
[INFO] time: 109.79ms
[INFO] time: 109.47ms
[INFO] time: 105.42ms
[INFO] time: 103.22ms
[INFO] time: 103.64ms
[INFO] time: 104.36ms
[INFO] time: 102.73ms
[INFO] time: 105.46ms
[INFO] time: 104.96ms
[INFO] time: 102.27ms
[INFO] time: 100.59ms
[INFO] time: 100.66ms
[INFO] time: 110.40ms
[INFO] time: 103.48ms
[INFO] time: 102.74ms
[INFO] time: 104.39ms
[INFO] time: 100.86ms
[INFO] time: 101.73ms
[INFO] time: 102.96ms
[INFO] time: 101.71ms
[INFO] time: 98.52ms
[INFO] time: 102.48ms


## 2 - 2. 스티커앱을 실행하고 카메라를 고정하고 서서히 멀어져봅니다. 혹은 아주 가까이 다가가 봅니다. 얼굴을 찾지 못하는 거리를 기록해주세요.

약 25 cm ~ 240 cm 거리까지 인식이 가능하였다. 최소한 얼굴 전체가 카메라에는 들어와야 작동이 되었으며, 240 cm 이상의 거리에서는 작동하지 못 했다.

## 2 - 3. 다시 자리로 돌아온 후 고개를 상하좌우로 움직여주세요. yaw, pitch, roll 각도의 개념을 직접 실험해 보고 각각 몇 도까지 정상적으로 스티커앱이 동작하는지 기록해주세요.

+ yaw : 시선을 기준으로 약 45 도
+ pitch : 시선을 기준으로 약 45 도
+ roll : 시선을 기준으로 약 45도



<br></br>
+ 참고 : 일반적인 허용범위

    + yaw : y축 기준 회전 → 높이 축 /  -45 ~ 45도
    + picth : x축 기준 회전 → 좌우 축 / -20 ~ 30도
    + roll : z축 기준 회전 → 거리 축 / -45 ~ 45도

## 2- 4. 만들고 싶은 스티커앱의 스펙(허용 거리, 허용 인원 수, 허용 각도, 안정성)을 정해주세요.

+ 거리 : 30 ~ 1 m 

+ 인원 : 최대 4인

+ 허용각도 : 하기 각도 이상에서는 자세가 불편함은 물론 화면을 바라보기 힘든 각도라고 생각한다.
    + yaw : 45 도
    + pitch : 45 도
    + roll : 45 도
    
+ 안정성 : 위 조건을 만족하면서 스티거가 움직이지 않는것을 목표로 한다.
               가만히 정면을 응시하고 있어도 스티커의 떨림 및 움직임이 존재한다.


<br></br>
+ 예 : 
    + 거리 : 25cm ~ 1m → 너무 가까우면 스티커의 의미가 없음, 셀카봉을 들었을 때의 유효거리
    
    + 인원 수 : 4명 → 4인 가족 기준
    
    + 허용 각도 : pitch : -20 ~ 30도, yaw : -45 ~ 45도, roll : -45 ~ 45도 → 화면을 바라볼 수 있는 각도
    
    + 안정성 : 위 조건을 만족하면서 FPPI (false positive per image) 기준 < 0.003, MR (miss rate) < 1 300장당 1번 에러 = 10초=30*10에 1번 에러

# 프로젝트 (3) 스티커 Out Bound 예외처리 하기

## 3 - 1. 지금까지 만든 스티커앱을 이용해서 예외 상황을 찾아주세요. 특히 서서히 영상에서 좌우 경계 밖으로 나가며 코드의 행동을 확인해 보세요.

웹캠 기반 스티커앱 실행시 아무런 에러코드 없이 `프로그램이 응답하지 않습니다.`라고 화면에 뜨며 `강제종료` 혹은 `기다리기`를 선택하는 에러 창이 종종 발생한다. 
    - 웹캠의 문제라고 생각하기엔 정상적으로 실행되는 경우도 많아 정확한 원인을 알 수가 없었다. (재부팅, 커널 Restart 를 해보아도 발생하였다.)

아래와 같은 오류가 발생하여 프로그램 실행이 되지 않거나 실행 중 종료된다.
    - 코드 문제인지, 코드 문제라면 왜 항상 발생되지 않는지 잘 모르겠다.
    
> Error Code
```python
error: OpenCV(4.5.1) /tmp/pip-req-build-hj027r8z/opencv/modules/core/src/arithm.cpp:666: error: (-209:Sizes of input arguments do not match) The operation is neither 'array op array' (where arrays have the same size and the same number of channels), nor 'array op scalar', nor 'scalar op array' in function 'arithm_op'
```

## 3 - 2. 문제가 어디에서 발생하는지 코드에서 확인합니다.

얼굴이 카메라 왼쪽 경계를 벗어나서 detection 되는 경우 refined_x 의 값이 음수가 됩니다.
img_bgr[..., refined_x:...] 에서 numpy array의 음수 index에 접근하게 되므로 예외가 발생합니다.

newaddsticker.py의 img2sticker 메소드에서 아래 부분을 수정해 주어야 합니다.

```python
### (이전 생략) ###

# sticker
img_sticker = cv2.resize(img_sticker, (w,h), interpolation=cv2.INTER_NEAREST)

refined_x = x - w // 2
refined_y = y - h

if refined_y < 0:
    img_sticker = img_sticker[-refined_y:]
    refined_y = 0

###
# TODO : x 축 예외처리 코드 추가
###

img_bgr = img_orig.copy()
sticker_area = img_bgr[refined_y:refined_y+img_sticker.shape[0], refined_x:refined_x+img_sticker.shape[1]]

img_bgr[refined_y:refined_y+img_sticker.shape[0], refined_x:refined_x+img_sticker.shape[1]] = \
    cv2.addWeighted(sticker_area, 1.0, img_sticker, 0.7, 0)

return img_bgr
```

## 3 - 3. Out bound 오류(경계 밖으로 대상이 나가서 생기는 오류)를 해결해 주세요.

위 예외처리 코드 부분에 들어가야 하는 코드는 아래와 같습니다. newaddsticker.py 파일을 수정해 주세요.

```python
    if refined_x < 0:
        img_sticker = img_sticker[:, -refined_x:]
        refined_x = 0
    elif refined_x + img_sticker.shape[1] >= img_orig.shape[1]:
        img_sticker = img_sticker[:, :-(img_sticker.shape[1]+refined_x-img_orig.shape[1])]
```

## 3 - 4. 다른 예외는 어떤 것들이 있는지 정의해 주세요. 어떤 것이 문제가 되는지 스스로 정해봅시다.



# 프로젝트 (4) 스티커앱 분석 - 거리, 인원 수, 각도, 시계열 안정성

## 4 - 1. 멀어지는 경우에 왜 스티커앱이 동작하지 않는지 분석해주세요. detection, landmark, blending 단계 중 무엇이 문제일까요?

dlib detection 이 문제입니다. 멀어지면 detector_hog 단계에서 bbox 가 출력되지 않습니다. 또한 Exploration 의 카메라 스티커앱 만들기때와 같이 너무 가깝거나 멀경우 Face Detection 이 안된다. 

```python
    # preprocess
    img_rgb = cv2.cvtColor(img_orig, cv2.COLOR_BGR2RGB)
    
    # detector
    img_rgb_vga = cv2.resize(img_rgb, (640, 360))
    dlib_rects = detector_hog(img_rgb_vga, 0)
    if len(dlib_rects) < 1:
        return img_orig
```

## 4 - 2. detector_hog의 문제를 해결하기 위해, 이미지 피라미드를 조절하여 성능을 향상시키는 간단한 방법이 있습니다. 이 방법을 활용하여 img2sticker 메소드를 간단히 고쳐 봅시다.

```python
def img2sticker(img_orig, img_sticker, detector_hog, landmark_predictor):
    # preprocess
    img_rgb = cv2.cvtColor(img_orig, cv2.COLOR_BGR2RGB)

    # detector
    img_rgb_vga = cv2.resize(img_rgb, (640, 360))
    dlib_rects = detector_hog(img_rgb_vga, 1) # <- 이미지 피라미드 수 변경
    if len(dlib_rects) < 1:
        return img_orig

    # landmark
    list_landmarks = []
    for dlib_rect in dlib_rects:
        points = landmark_predictor(img_rgb_vga, dlib_rect)
        list_points = list(map(lambda p: (p.x, p.y), points.parts()))
        list_landmarks.append(list_points)
```
<br></br>
수정 후에 webcam_sticker.py 를 다시한번 실행하여 스티커앱이 잘 실행되는지 확인해 봅시다.



## 4 - 3. 위에서 새롭게 시도한 방법의 문제점은 무엇인가요?

속도가 현저히 느려집니다. 기존 30ms/frame 에서 120ms/frame 으로 약 4배 느려짐 → 실시간 구동이 불가능 합니다.

## 4 - 4. 실행시간을 만족할 수 있는 방법을 찾아봅시다.
hog 디텍터를 딥러닝 기반 디텍터로 변경할 수 있습니다. hog 학습 단계에서 다양한 각도에 대한 hog 특징을 모두 추출해서 일반화 하기 어렵기 때문에 딥러닝 기반 검출기의 성능이 훨씬 좋습니다.

딥러닝 기반 detection 방법을 조사합니다. 아래 링크를 참고하면 도움이 될 것입니다.

+ 참고 : [How does the OpenCV deep learning face detector work?](https://www.pyimagesearch.com/2018/02/26/face-detection-with-opencv-and-deep-learning/)

opencv 는 intel cpu 을 사용할 때 dnn 모듈이 가속화를 지원하고 있습니다. 따라서 mobilenet 과 같은 작은 backbone 모델을 사용하고 ssd 를 사용한다면 충분히 만족할 만한 시간과 성능을 얻을 수 있습니다.

+ 참고 : [Facial Landmark Detection](https://www.learnopencv.com/facial-landmark-detection/)

## 4 - 5. 인원 수, 각도 등 각 문제에 대해서 1 - 4번을 반복해주세요. (정해진 답은 없습니다.)

자유롭게 설계해주세요. 각도 문제에 대해서는 아래 방법을 적용해볼 수 있습니다.

Facial Landmark Detection

# 프로젝트 (5) 칼만 필터 적용하기

## 5 - 1. 카메라 앞에서 가만히 있을 때 스티커의 움직임을 관찰해 주세요. 어떤 문제가 발생하나요?

가만히 있어도 스티커의 크리가 일정하게 유지되지 않고, 떨리는 것처럼 보이는 현상이 발생합니다.

## 5 - 2. 이론 강의에서 배운 칼만 필터를 적용해서 스티커 움직임을 안정화시켜 주세요.

칼만 필터를 구현한 모듈인 `kalman.py` 와 이를 이용하여 tracker를 구현한 `kpkf.py` 를 이용하여 칼만필터를 적용한 `webcam_sticker_kf.py`를 함께 첨부합니다. 실행해 보면 현재는 웹캠이 아니라 샘플 동영상에 칼만필터가 적용된 것을 확인하실 수 있습니다.

동영상 입력을 웹캠으로 바꾸면 우리가 만들고 있는 웹캠에도 동일하게 적용할 수 있습니다. `webcam_sticker_kf.py` 를 참고하여 자신만의 `webcam_sticker.py` 를 완성해 주세요.


In [1]:
import numpy as np
import cv2
import dlib

from addsticker_kf import img2sticker_kf

detector_hog = dlib.get_frontal_face_detector()
landmark_predictor = dlib.shape_predictor('./models/shape_predictor_68_face_landmarks.dat')

def main():
    cv2.namedWindow('show', 0)
    cv2.resizeWindow('show', 640, 360)

    vc = cv2.VideoCapture(0)    
    img_sticker = cv2.imread('./images/king.png')

    vlen = int(vc.get(cv2.CAP_PROP_FRAME_COUNT))
    print (vlen) # 웹캠은 video length 가 0 입니다.

    # 정해진 길이가 없기 때문에 while 을 주로 사용합니다.
    # for i in range(vlen):
    while True:
        ret, img = vc.read()
        if ret == False:
            break
        start = cv2.getTickCount()
        img = cv2.flip(img, 1)  # 보통 웹캠은 좌우 반전

        # 스티커 메소드를 사용
        img_result = img2sticker_kf(img, img_sticker.copy(), detector_hog, landmark_predictor)   

        time = (cv2.getTickCount() - start) / cv2.getTickFrequency() * 1000
        print ('[INFO] time: %.2fms'%time)
        
        cv2.imshow('show', img_result)
        key = cv2.waitKey(1)
        if key == 27:
            break


if __name__ == '__main__':
    main()

-1
[INFO] time: 30.12ms
[INFO] time: 31.57ms
[INFO] time: 30.86ms
[INFO] time: 29.65ms
[INFO] time: 30.55ms
[INFO] time: 27.87ms
[INFO] time: 28.28ms
[INFO] time: 30.11ms
[INFO] time: 29.25ms
[INFO] time: 27.37ms
[INFO] time: 35.71ms
[INFO] time: 27.72ms
[INFO] time: 29.47ms
[INFO] time: 28.38ms
[INFO] time: 32.53ms
[INFO] time: 29.27ms
[INFO] time: 38.34ms
[INFO] time: 28.11ms
[INFO] time: 30.05ms
[INFO] time: 27.43ms
[INFO] time: 29.17ms
[INFO] time: 30.31ms
[INFO] time: 26.44ms
[INFO] time: 28.01ms
[INFO] time: 26.09ms
[INFO] time: 24.26ms
[INFO] time: 26.89ms
[INFO] time: 24.98ms
[INFO] time: 28.80ms
[INFO] time: 31.32ms
[INFO] time: 27.64ms
[INFO] time: 24.95ms
[INFO] time: 27.07ms
[INFO] time: 28.65ms
[INFO] time: 29.34ms
[INFO] time: 27.93ms
[INFO] time: 30.02ms
[INFO] time: 27.08ms
[INFO] time: 27.76ms
[INFO] time: 27.69ms
[INFO] time: 29.35ms
[INFO] time: 29.98ms
[INFO] time: 29.27ms
[INFO] time: 27.45ms
[INFO] time: 29.76ms
[INFO] time: 27.70ms
[INFO] time: 29.02ms
[INFO] tim